In [1]:
import sys
import numpy as np

sys.path.append('..')

In [2]:
import core
from core import NPerson, Region

## 換気設備

旧評価方法

In [3]:
def get_std_consumption_for_ventilation(number_of_people, total_floor_area, ventilation_coeff):
    
    # general ventilation
    c = 33.9 # MJ/m2
    anual_general = c * total_floor_area # for general ventilation
    
    # local ventilation
    anual_local = {1 : 130.0, 2 : 259.0, 3 : 389.0, 4 : 518.0}[number_of_people]
    
    # divide to montly consumption
    month_ratios = [8.5, 7.7, 8.5, 8.2, 8.5, 8.2, 8.5, 8.5, 8.2, 8.5, 8.2, 8.5]
    return [month_ratio / 100 * (anual_general + anual_local) * ventilation_coeff for month_ratio in month_ratios]

比較

In [4]:
core.get_e_v_ref_p_m(a_a=120.0, p=NPerson.ONE)

array([356.152648 , 321.644704 , 356.152648 , 344.5014528, 355.833496 ,
       344.6279424, 356.2188208, 356.1195616, 344.5014528, 355.993072 ,
       344.6941152, 355.993072 ])

In [5]:
np.array(get_std_consumption_for_ventilation(number_of_people=1, total_floor_area=120.0, ventilation_coeff=1.0)).round(2)

array([356.83, 323.25, 356.83, 344.24, 356.83, 344.24, 356.83, 356.83,
       344.24, 356.83, 344.24, 356.83])

## 照明設備

旧評価方法

In [6]:
def get_std_consumption_for_lighting(number_of_people,
                                     main_habitable_room_floor_area, other_habitable_room_floor_area, total_floor_area,
                                     lighting_coeff
                                    ):
    
    non_habitable_room_floor_area = total_floor_area - main_habitable_room_floor_area - other_habitable_room_floor_area
    
    a_MR, b_MR = {
        1: (28.4,  417.2),
        2: (58.0,  885.8),
        3: (65.7, 1002.3),
        4: (73.5, 1118.9)
    }[number_of_people]
    
    a_OR, b_OR = {
        1: ( 1.7,  38.5),
        2: ( 3.3,  78.0),
        3: (15.8, 432.1),
        4: (28.2, 786.2)
    }[number_of_people]

    a_NO, b_NO = {
        1: ( 3.2, 193.0),
        2: ( 7.8, 458.0),
        3: ( 9.7, 580.0),
        4: (11.6, 701.0)
    }[number_of_people]

    anual_MR = a_MR * main_habitable_room_floor_area + b_MR
    anual_OR = a_OR * other_habitable_room_floor_area + b_OR
    anual_NO = a_NO * non_habitable_room_floor_area + b_NO
    
    # divide to montly consumption
    month_ratios = [8.5, 7.7, 8.5, 8.2, 8.3, 8.2, 8.7, 8.5, 8.2, 8.4, 8.4, 8.4]
    
    return [month_ratio / 100 * (anual_MR + anual_OR + anual_NO) * lighting_coeff for month_ratio in month_ratios]

比較

In [7]:
result = core.get_e_l_ref_p_m(a_a=120.08, a_mr=29.81, a_or=51.34, p=NPerson.FOUR)
result, result.sum()

(array([572.79476, 513.66573, 572.79476, 546.68775, 555.83696, 548.79275,
        585.54015, 566.40716, 546.68775, 564.25082, 561.53814, 564.25082]),
 6699.24755)

In [8]:
result = np.array(
    get_std_consumption_for_lighting(number_of_people=4, main_habitable_room_floor_area=29.81, other_habitable_room_floor_area=51.34, total_floor_area=120.08, lighting_coeff=1.0)
).round(3)
result, sum(result)

(array([569.203, 515.631, 569.203, 549.114, 555.81 , 549.114, 582.596,
        569.203, 549.114, 562.507, 562.507, 562.507]),
 6696.508999999998)

## 家電

旧評価方法

In [9]:
def get_std_consumption_for_electric_appliance(number_of_people, electric_appliance_coeff):
    
    anual = {1:9965.0, 2:12086.0, 3:16523.0, 4:17388}[number_of_people]
    
    month_ratios = [8.5, 7.7, 8.5, 8.2, 8.3, 8.2, 8.7, 8.5, 8.2, 8.4, 8.4, 8.4]
    
    return [month_ratio / 100 * anual * electric_appliance_coeff for month_ratio in month_ratios]

比較

In [10]:
np.array(get_std_consumption_for_electric_appliance(1, 1.0)).round(1), \
np.array(get_std_consumption_for_electric_appliance(2, 1.0)).round(1), \
np.array(get_std_consumption_for_electric_appliance(3, 1.0)).round(1), \
np.array(get_std_consumption_for_electric_appliance(4, 1.0)).round(1)


(array([847. , 767.3, 847. , 817.1, 827.1, 817.1, 867. , 847. , 817.1,
        837.1, 837.1, 837.1]),
 array([1027.3,  930.6, 1027.3,  991.1, 1003.1,  991.1, 1051.5, 1027.3,
         991.1, 1015.2, 1015.2, 1015.2]),
 array([1404.5, 1272.3, 1404.5, 1354.9, 1371.4, 1354.9, 1437.5, 1404.5,
        1354.9, 1387.9, 1387.9, 1387.9]),
 array([1478. , 1338.9, 1478. , 1425.8, 1443.2, 1425.8, 1512.8, 1478. ,
        1425.8, 1460.6, 1460.6, 1460.6]))

In [11]:
core.get_e_app_ref_p_m(p=NPerson.ONE), \
core.get_e_app_ref_p_m(p=NPerson.TWO), \
core.get_e_app_ref_p_m(p=NPerson.THREE), \
core.get_e_app_ref_p_m(p=NPerson.FOUR)

(array([849.7, 765.3, 849.7, 815.6, 836.1, 819.9, 854.6, 847.2, 815.6,
        842.9, 824.9, 842.9]),
 array([1031.2,  928.7, 1031.2,  988.7, 1012.5,  995.1, 1037.2, 1028.3,
         988.7, 1021.9, 1001. , 1021.9]),
 array([1409.4, 1268.9, 1409.4, 1352. , 1384.8, 1359.1, 1419.7, 1404.2,
        1352. , 1397.1, 1369.5, 1397.1]),
 array([1481.9, 1333.7, 1481.9, 1424. , 1459.5, 1427.1, 1498. , 1473.8,
        1424. , 1470.7, 1443.2, 1470.7]))

## 調理

旧評価方法

In [12]:
def get_std_consumption_for_cooking(number_of_people, cooking_coeff):
    
    anual = {1:2217.0, 2:2724.0, 3:3288.0, 4:3852}[number_of_people]
    
    month_ratios = [8.6, 7.8, 8.6, 8.1, 8.1, 8.4, 8.6, 8.6, 8.1, 8.4, 8.3, 8.4]
    
    return [month_ratio / 100 * anual * cooking_coeff for month_ratio in month_ratios]

比較

In [13]:
np.array(get_std_consumption_for_cooking(1, 1.0)).round(1), \
np.array(get_std_consumption_for_cooking(2, 1.0)).round(1), \
np.array(get_std_consumption_for_cooking(3, 1.0)).round(1), \
np.array(get_std_consumption_for_cooking(4, 1.0)).round(1)

(array([190.7, 172.9, 190.7, 179.6, 179.6, 186.2, 190.7, 190.7, 179.6,
        186.2, 184. , 186.2]),
 array([234.3, 212.5, 234.3, 220.6, 220.6, 228.8, 234.3, 234.3, 220.6,
        228.8, 226.1, 228.8]),
 array([282.8, 256.5, 282.8, 266.3, 266.3, 276.2, 282.8, 282.8, 266.3,
        276.2, 272.9, 276.2]),
 array([331.3, 300.5, 331.3, 312. , 312. , 323.6, 331.3, 331.3, 312. ,
        323.6, 319.7, 323.6]))

In [14]:
core.get_e_cc_ref_p_m(p=NPerson.ONE), \
core.get_e_cc_ref_p_m(p=NPerson.TWO), \
core.get_e_cc_ref_p_m(p=NPerson.THREE), \
core.get_e_cc_ref_p_m(p=NPerson.FOUR)

(array([191. , 171.1, 191. , 179.5, 180.6, 183.6, 193.1, 189.9, 179.5,
        185.8, 185.8, 185.8]),
 array([235.3, 211.3, 235.3, 220.1, 221.2, 227.6, 234.3, 235.7, 220.1,
        228.2, 226.6, 228.2]),
 array([284. , 255.1, 284. , 265.7, 267. , 274.7, 282.8, 284.6, 265.7,
        275.5, 273.6, 275.5]),
 array([332.7, 298.8, 332.7, 311.2, 312.8, 321.9, 331.4, 333.4, 311.2,
        322.8, 320.5, 322.8]))